In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("data/itineraries_snappy.parquet", engine="pyarrow", columns=[ "searchDate", "flightDate", "startingAirport", "destinationAirport","travelDuration", "isBasicEconomy", "isRefundable", "isNonstop", "totalFare", "seatsRemaining", "travelDistance","segmentsDepartureTimeRaw", "segmentArrivalTimeRaw", "segmentsAirlineCode", "segmentsCabinCode" ])

In [3]:
df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


In [16]:
df.searchDate = pd.to_datetime(df.searchDate)
df.flightDate = pd.to_datetime(df.flightDate)

In [ ]:
df['minutes'] = (
    df['travel_duration']
    .str.extract(r'PT(?:(\d+)H)?(?:(\d+)M)?')
    .fillna(0)
    .astype(int)
    .apply(lambda x: x[0] * 60 + x[1], axis=1)
)

In [ ]:
df.head()

In [5]:
average_seats_remaining = df[('seatsRemaining')].mean()
print(average_seats_remaining)

5.976311522528228


In [15]:
df.segmentsDepartureAirportCode.unique()

array(['ATL', 'ATL||CLT', 'ATL||IAD', ..., 'DTW||ITH', 'JFK||ROC||BOS',
       'ONT||DEN||MCO'], dtype=object)

In [13]:
df.segmentsCabinCode.unique()

array(['coach', 'coach||coach', 'coach||coach||coach||coach',
       'coach||coach||coach', 'first', 'premium coach', 'first||coach',
       'business', 'premium coach||premium coach', 'first||first',
       'coach||premium coach', 'coach||first', 'first||coach||coach',
       'business||first||first', 'business||coach||coach',
       'business||coach', 'business||business||coach',
       'coach||business||business', 'business||coach||business',
       'premium coach||premium coach||coach', 'coach||business',
       'coach||coach||business', 'business||business',
       'coach||coach||first||coach', 'premium coach||coach',
       'first||first||first', 'coach||first||first',
       'coach||coach||first', 'coach||first||coach',
       'coach||business||coach', 'coach||coach||premium coach',
       'premium coach||coach||coach', 'first||first||coach',
       'coach||premium coach||coach',
       'coach||coach||coach||premium coach', 'business||first',
       'premium coach||business||coa

In [14]:
mask_mixed_cabin = df['segmentsCabinCode'].str.contains(r'first.*\|\|.*(coach|business|premium coach)|'
                                                         r'coach.*\|\|.*(first|business|premium coach)|'
                                                         r'business.*\|\|.*(first|coach|premium coach)|'
                                                         r'premium coach.*\|\|.*(first|coach|business)',
                                                         regex=True)

percent_mixed_cabin = (mask_mixed_cabin.sum() / len(df)) * 100

print(f"Percentage of entries with mixed cabins: {percent_mixed_cabin:.2f}%")

/var/folders/k2/h0jvj1b548l9_206_d8lb3yw0000gn/T/ipykernel_98555/554970052.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_mixed_cabin = df['segmentsCabinCode'].str.contains(r'first.*\|\|.*(coach|business|premium coach)|'


Percentage of entries with mixed cabins: 0.14%
